Изменение кода: измените значение 'files_name' на директорию, где находится папка 'annotation system', например: './annotation system/'

Перед загрузкой файлов необходимо переименовать назначенный вам файл txt в 'random_fts_files.txt'

In [ ]:
import requests 
import os
import time

# Set file paths
files_name ='your file path/annotation system/' # Change to the directory where your annotation system folder is located
links_file = files_name + 'random_fts_files.txt'  # File containing download links
save_dir = files_name + 'fts images'  # Save directory

def clean_filename(filename):
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

def download_files_from_list(links_file, save_dir):
    # Create save directory
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")
    
    # Read links file
    with open(links_file, 'r') as file:
        urls = [line.strip() for line in file if line.strip()]
    
    total_files = len(urls)
    print(f"Total files to download: {total_files}")
    
    # Download files
    success_count = 0
    error_count = 0
    
    for i, url in enumerate(urls):
        try:
            # Get filename
            filename = os.path.basename(url)
            clean_name = clean_filename(filename)
            file_path = os.path.join(save_dir, clean_name)
            
            # Check if file already exists
            if os.path.exists(file_path):
                print(f"[{i+1}/{total_files}] File already exists: {clean_name}")
                success_count += 1
                continue
            
            # Download file
            print(f"[{i+1}/{total_files}] Downloading: {clean_name}")
            response = requests.get(url, stream=True)
            
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                success_count += 1
                print(f"✓ Successfully downloaded: {clean_name}")
            else:
                error_count += 1
                print(f"× Download failed: {url}, Status code: {response.status_code}")
                
            # Time interval to avoid requesting too quickly
            if (i + 1) % 10 == 0:
                time.sleep(1)
                
        except Exception as e:
            error_count += 1
            print(f"× Download error: {url}, Error: {str(e)}")
    
    print(f"\nDownload complete! Success: {success_count}, Failed: {error_count}, Total: {total_files}")

# Start downloading
download_files_from_list(links_file, save_dir)
